In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text[:8]

'First Ci'

In [5]:
len(text)

1115394

In [10]:
chars = sorted(list(set(''.join(text))))
stoi = {c : i for i, c in enumerate(chars)}
itos = {i : c for c, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
vocab_size = len(itos)
print(itos)
print(vocab_size)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
65


In [28]:
# train and test split
data = torch.tensor(encode(text), dtype=torch.long)
n_train = int(0.9*len(data))

train_data = data[:n_train]       # 90%
eval_data = data[n_train:]         # 10%
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [29]:
# hyperparameters
batch_size = 32 # how many independtent sequences will we process in parallel?
block_size = 8 # what is the maxmium context length for prediction?
epochs = 3000 
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [30]:
def get_batch(split):
    # generate small batch of data of inputs data X and targets y
    data = train_data if split == 'train' else eval_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

In [39]:
X, y = get_batch('eval')
X[:2], y[:2]

(tensor([[ 1, 39,  1, 58, 61, 47, 52, 49],
         [53, 56, 61, 39, 56, 42,  6,  1]]),
 tensor([[39,  1, 58, 61, 47, 52, 49,  1],
         [56, 61, 39, 56, 42,  6,  1, 21]]))

In [31]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, X, y=None):
        # X and y are both (B, T) tensor integers, B = batch_size, T = block_size
        logits = self.token_embedding_table(X) # (B, T, C)
        
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits, y)
        return logits, loss
    
    def generate(self, X, max_new_tokens):
        # X is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(X)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_nxt = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            X = torch.cat((X, idx_nxt), dim=1) # (B, T+1)
        return X
        

In [33]:
model = BigramLanguageModel(vocab_size)
model = model.to(device)

In [34]:
print('parameters number:', sum(p.nelement() for p in model.parameters()))

parameters number: 4225


In [36]:
# train model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(epochs):
    # every once in a while evaluate the loss on train and eval sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, eval loss {losses['eval']:.4f}")
    
    # sample a batch of data
    Xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(Xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7339, eval loss 4.7320
step 300: train loss 2.8305, eval loss 2.8517
step 600: train loss 2.5466, eval loss 2.5730
step 900: train loss 2.5092, eval loss 2.5105
step 1200: train loss 2.4877, eval loss 2.5034
step 1500: train loss 2.4768, eval loss 2.4950
step 1800: train loss 2.4766, eval loss 2.4836
step 2100: train loss 2.4754, eval loss 2.4838
step 2400: train loss 2.4523, eval loss 2.4848
step 2700: train loss 2.4536, eval loss 2.4922


In [37]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


Thyowasord h thy fanotctag
Fohinampy oan'TOLLOf ghay

Ty lie w; be; n, hieglaly om.
bale?
ycckiserest al thomyomerdik wive airme werearingQ k. st,
LAlinchert bbuave
ze.
Noo be.
Anoupaly of so, vyshes, ye hingrt 'e o oroup win Thes, bu:
Tehel helve ANTRWhewin.
LEOMESeme.
R:
ds nte trto Y Wh met whencoryee ce:

MELe therreringely bunol t tlige waty t m. g bheanove pe tinovimu w
ONGoe t bory bllle hite or thind hooy nt.
INCHenceagind f s.
Sogr ongindou,

Anonecethe t m,
I gngeoupou, win hall dor st
